# Factorizacion LU

## Códigos en Python
Escribe en Python un codigo que obtenga la factorización LU otro que obtenga la factorizacion PLU y otro que obtenga la factorizacion de Cholesky de una matriz A. En caso de que la factorizacion no pueda realizarse sobre la matriz A, tu codigo debe arrojar que es lo que la matriz A no cumple.

In [176]:
import numpy as np
import matplotlib.pyplot as plt

In [177]:
def lu_fact(A: np.ndarray, epsilon: float = 1e-15) -> tuple[np.ndarray, np.ndarray]:
    m, n = np.shape(A)
    if m != n:
        raise InvalidInputMatrix('La matriz no es cuadrada.')

    L = np.eye(n)  # Inicializamos L como la matriz identidad
    U = np.copy(A)  # Copiamos A en U
    
    for j in range(n - 1):
        if abs(U[j, j]) < epsilon:
            raise InvalidInputMatrix('La matriz no es invertible.')
        
        for i in range(j + 1, n):
            L[i, j] = U[i, j] / U[j, j]  # Guardamos los factores de eliminación en L
            U[i] = U[i] - L[i, j] * U[j]  # Realizamos la eliminación en U (hace cero debajo de la diagonal)
    
    return L, U


In [178]:
def swap_rows(M: np.ndarray, row1: int, row2: int) -> np.ndarray:
    # Intercambio de filas
    M[[row1, row2]] = M[[row2, row1]]
    return M

def max_index(column: np.ndarray, start: int) -> int:
    return np.argmax(np.abs(column[start:])) + start

def plu_fact(A: np.ndarray, epsilon: float = 1e-15) -> tuple[np.ndarray, np.ndarray, np.ndarray]:
    m, n = np.shape(A)
    if m != n:
        raise InvalidInputMatrix('La matriz no es cuadrada.')
    
    P = np.eye(n)
    L = np.zeros((n, n))
    U = np.copy(A)
    
    for j in range(n - 1):
        # Encontrar el índice de la fila con el mayor valor absoluto en la columna j
        index = max_index(U[:, j], j)
        
        if abs(U[index, j]) < epsilon:
            raise InvalidInputMatrix('La matriz no es invertible.')
        
        # Intercambiar filas en P, L y U
        P = swap_rows(P, j, index)
        L = swap_rows(L, j, index)
        U = swap_rows(U, j, index)
        
        for i in range(j + 1, n):
            L[i, j] = U[i, j] / U[j, j]
            U[i] = U[i] - L[i, j] * U[j]
    
    # Añadir la identidad a L
    L = L + np.eye(n)
    
    return P, L, U


In [179]:
class InvalidInputMatrix(Exception):
    pass

def cholesky_fact(A: np.ndarray, epsilon: float = 1e-15) -> np.ndarray:
    m, n = np.shape(A)
    if m != n:
        raise InvalidInputMatrix('La matriz no es cuadrada.')
    
    # Inicializar matriz U (triangular superior)
    U = np.zeros_like(A)
    
    for i in range(n):
        for j in range(i, n):
            if i == j:
                suma = np.sum(U[i, :i]**2)
                diag_value = A[i, i] - suma
                if diag_value < epsilon:
                    raise InvalidInputMatrix('La matriz no es definida positiva.')
                U[i, i] = np.sqrt(diag_value)
            else:
                suma = np.sum(U[i, :i] * U[j, :i])
                U[j, i] = (A[i, j] - suma) / U[i, i]
    
    return U.T  # Se devuelve la matriz triangular superior


# Error en la factorizacion

## A) y B)

In [215]:
# calcular el error de descomposición LU
def lu_error(A: np.ndarray, L: np.ndarray, U: np.ndarray) -> float:
    return np.linalg.norm(A - L @ U)

#  calcular el error de descomposición PLU
def plu_error(A: np.ndarray, P: np.ndarray, L: np.ndarray, U: np.ndarray) -> float:
    return np.linalg.norm(P @ A - L @ U)

#calcular el error de descomposición de Cholesky
def cholesky_error(A: np.ndarray, U: np.ndarray) -> float:
    return np.linalg.norm(A - U.T @ U)

scale = 25
N = np.arange(2, 503, 10)

# Inicialización de los arrays para los errores
lu_errors_arr = np.array([])
plu_errors_arr = np.array([])
cholesky_errors_arr = np.array([])

# Bucle para calcular los errores en cada descomposición
for n in N:
    # Generamos una matriz aleatoria de tamaño (n, n)
    M = scale * np.random.rand(n, n)
    
    # Descomposición LU
    L, U = lu_fact(M)  # Asegúrate de que tu función lu_fact retorne L y U
    lu_errors_arr = np.append(lu_errors_arr, lu_error(M, L, U))
    
    # Descomposición PLU
    P, L, U = plu_fact(M)  # Asegúrate de que plu_fact retorne P, L, y U
    plu_errors_arr = np.append(plu_errors_arr, plu_error(M, P, L, U))
    
    # Descomposición de Cholesky (requiere una matriz simétrica y definida positiva)
    A = M @ M.T  # Asegura que sea simétrica positiva para Cholesky
    U = cholesky_fact(A)  # La factorización de Cholesky devuelve U (matriz triangular superior)
    cholesky_errors_arr = np.append(cholesky_errors_arr, cholesky_error(A, U))

# Imprimir resultados de errores
print("Errores LU:", lu_errors_arr)
print("Errores PLU:", plu_errors_arr)
print("Errores Cholesky:", cholesky_errors_arr)


## C)

In [221]:
plt.figure(figsize=(10,20))

# Graficar los errores para LU, PLU y Cholesky
plt.plot(N, plu_errors_arr,'b--o', label='LU', markersize=6, linewidth=1.5)
plt.plot(N, lu_errors_arr, 'r--s', label='PLU', markersize=6, linewidth=1.5)
plt.plot(N, cholesky_errors_arr, 'g--^', label='Cholesky', markersize=6, linewidth=1.5)

# Etiquetas de los ejes
plt.xlabel('$dim(M)=n_i$', fontsize=14)
plt.ylabel('Error (Norma de Frobenius)', fontsize=14)

# Título de la gráfica
plt.title('Errores de Factorización en Función de la Dimensión (Escala Logarítmica)', fontsize=16)

# Mostrar la cuadrícula con mayor detalle
plt.grid(alpha=0.6)

# Añadir la leyenda para identificar las curvas
plt.legend(fontsize=12)

# Mostrar la gráfica
plt.show()


## D)

In [220]:
# Definir las matrices de errores 
lu_errors_matrix = np.zeros((25, len(N)))
plu_errors_matrix = np.zeros((25, len(N)))
cholesky_errors_matrix = np.zeros((25, len(N)))

# Definir factor de escala
scale = 25

# Iterar sobre cada dimensión n_i en N
for j, n in enumerate(N):
    for i in range(25):  # Generar 25 matrices aleatorias para cada dimensión n_i
        # Matriz aleatoria de dimensión n x n
        M = scale * np.random.rand(n, n)
        
        try:
            # Calcular factorizaciones y errores para LU
            L, U = lu_fact(M)
            lu_errors_matrix[i, j] = lu_error(M, L, U)
            
            # Calcular factorizaciones y errores para PLU
            P, L, U = plu_fact(M)
            plu_errors_matrix[i, j] = plu_error(M, P, L, U)
            
            # Calcular factorización y error para Cholesky (Matriz positiva definida)
            A = M @ M.T  # A = M * M^T para garantizar que sea positiva definida
            U = cholesky_fact(A)
            cholesky_errors_matrix[i, j] = cholesky_error(A, U)
        
        except InvalidInputMatrix:
            # En caso de error, asignar NaN
            lu_errors_matrix[i, j] = np.nan
            plu_errors_matrix[i, j] = np.nan
            cholesky_errors_matrix[i, j] = np.nan

# Calcular la media y desviación estándar para cada dimensión n_i
lu_mean = np.nanmean(lu_errors_matrix, axis=0)
lu_std = np.nanstd(lu_errors_matrix, axis=0)

plu_mean = np.nanmean(plu_errors_matrix, axis=0)
plu_std = np.nanstd(plu_errors_matrix, axis=0)

cholesky_mean = np.nanmean(cholesky_errors_matrix, axis=0)
cholesky_std = np.nanstd(cholesky_errors_matrix, axis=0)

# Graficar la media de los errores con barras de error (desviación estándar)
plt.figure(figsize=(10, 6))

# Usar una escala logarítmica para los valores de error en el eje Y
plt.yscale('log')

# Graficar las medias y desviaciones estándar de LU
plt.errorbar(N, lu_mean, yerr=lu_std, fmt='b--o', label='LU', markersize=6, linewidth=1.5)

# Graficar las medias y desviaciones estándar de PLU
plt.errorbar(N, plu_mean, yerr=plu_std, fmt='r--s', label='PLU', markersize=6, linewidth=1.5)

# Graficar las medias y desviaciones estándar de Cholesky
plt.errorbar(N, cholesky_mean, yerr=cholesky_std, fmt='g--^', label='Cholesky', markersize=6, linewidth=1.5)

# Etiquetas de los ejes
plt.xlabel('$dim(M)=n_i$', fontsize=14)
plt.ylabel('Error Promedio (Norma de Frobenius)', fontsize=14)

# Título de la gráfica
plt.title('Media y Desviación Estándar de los Errores en Función de la Dimensión', fontsize=16)

# Mostrar la cuadrícula
plt.grid(alpha=0.6)

# Añadir la leyenda para identificar las curvas
plt.legend(fontsize=12)

# Mostrar la gráfica
plt.show()


# Imprimir los resultados para cada método
print("Media y desviación estándar de LU")
print(lu_mean)
print(lu_std)

print("\nMedia y desviación estándar de PLU")
print(plu_mean)
print(plu_std)

print("\nMedia y desviación estándar de Cholesky")
print(cholesky_mean)
print(cholesky_std)


## E) 

In [233]:

# Graficar la media de los errores para LU, PLU y Cholesky en función de la dimensión
plt.figure(figsize=(10, 6))

# Graficar las medias de los errores
plt.plot(N, lu_mean, 'b--o', label='LU (Media de Errores)', markersize=6, linewidth=1.5)
plt.plot(N, plu_mean, 'r--s', label='PLU (Media de Errores)', markersize=6, linewidth=1.5)
plt.plot(N, cholesky_mean, 'g--^', label='Cholesky (Media de Errores)', markersize=6, linewidth=1.5)

# Etiquetas de los ejes
plt.xlabel('$dim(M)=n_i$', fontsize=14)
plt.ylabel('Error Promedio (Norma de Frobenius)', fontsize=14)

# Título de la gráfica
plt.title('Media de Errores de Factorización en Función de la Dimensión', fontsize=16)

# Mostrar la cuadrícula
plt.grid(alpha=0.6)

# Añadir la leyenda para identificar las curvas
plt.legend(fontsize=12)

# Mostrar la gráfica
plt.show()


### Análisis de la Estabilidad de Métodos Numéricos Según Promedio de Errores

#### 1. Método **PLU** (Descomposición con pivoteo):
- **Estabilidad**: PLU es el más estable de los métodos debido a la inclusión de pivoteo, que ayuda a manejar mejor matrices mal condicionadas.
- **Comportamiento del error**: El crecimiento del error es controlado y más lento, lo que se refleja en un promedio de error más bajo conforme aumenta la dimensión.
- **Conclusión**: El pivoteo limita la propagación de errores, haciendo que PLU sea más robusto frente a matrices complejas o de gran tamaño.

#### 2. Método **LU** (Descomposición sin pivoteo):
- **Inestabilidad**: LU es más propenso a la inestabilidad numérica, ya que no cuenta con un pivoteo que corrija las entradas problemáticas de la matriz.
- **Comportamiento del error**: El error crece más rápido que en PLU debido a la falta de control sobre los valores pequeños o mal condicionados.
- **Conclusión**: LU muestra un incremento mayor en los errores con el aumento de la dimensión, reflejando su menor estabilidad.

#### 3. Método **Cholesky**:
- **Requerimientos estrictos**: Solo funciona con matrices simétricas y definidas positivas, lo que lo hace más sensible a cualquier desviación de estas condiciones.
- **Inestabilidad**: Cualquier violación de las condiciones requeridas lleva a una rápida acumulación de errores.
- **Comportamiento del error**: El promedio de error crece de forma más agresiva que en PLU, siendo comparable o incluso mayor que LU en ciertas dimensiones.
- **Conclusión**: Cholesky es menos predecible y más propenso a errores en grandes dimensiones, especialmente si las condiciones de la matriz no se cumplen rigurosamente.

#### **Conclusión General**:
- **PLU** es el más estable debido a su técnica de pivoteo.
- **LU** y **Cholesky** muestran un crecimiento de error más rápido, siendo Cholesky el más inestable cuando las condiciones ideales no se cumplen.


## F)

In [223]:
# Graficar la desviación estándar de los errores para LU y PLU en función de la dimensión
plt.figure(figsize=(10, 6))

# Graficar las desviaciones estándar
plt.plot(N, lu_std, 'b--o', label='LU (Desviación Estándar)', markersize=6, linewidth=1.5)
plt.plot(N, plu_std, 'r--s', label='PLU (Desviación Estándar)', markersize=6, linewidth=1.5)

# Etiquetas de los ejes
plt.xlabel('$dim(M)=n_i$', fontsize=14)
plt.ylabel('Desviación Estándar del Error', fontsize=14)

# Título de la gráfica
plt.title('Desviación Estándar de los Errores en Función de la Dimensión', fontsize=16)

# Mostrar la cuadrícula
plt.grid(alpha=0.6)

# Añadir la leyenda para identificar las curvas
plt.legend(fontsize=12)

# Mostrar la gráfica
plt.show()


### Análisis de la Desviación Estándar de los Errores en LU y PLU

#### **LU**:
- La desviación estándar de los errores en la factorización LU **aumenta significativamente** conforme crece la dimensión. Esto indica que el método LU se vuelve menos predecible y más inestable para matrices de mayor tamaño.
- La falta de pivoteo en LU amplifica los errores numéricos, lo que genera mayor variabilidad en los resultados.

#### **PLU**:
- En comparación, la desviación estándar de los errores en PLU **permanece mucho más baja** y estable conforme crece la dimensión. El pivoteo permite controlar los errores numéricos y evita que se propaguen, lo que reduce la variabilidad en los errores.
- Aunque hay un pequeño incremento en la desviación estándar para dimensiones más grandes, este es mucho menos pronunciado que en LU, lo que confirma que **PLU es más estable** en general.

#### **Conclusión**:
- **PLU** muestra una desviación estándar considerablemente menor, lo que indica que es más confiable para el uso en matrices de mayor dimensión. La estabilidad que proporciona el pivoteo en PLU lo convierte en una opción superior frente a LU, especialmente en problemas de gran escala.


## G

In [236]:
# Calcular la media total de los errores para cada algoritmo
lu_mean_total = np.nanstd(lu_mean)  # Promedio de errores LU
plu_mean_total = np.nanstd(plu_mean)  # Promedio de errores PLU
cholesky_mean_total = np.nanstd(cholesky_mean)  # Promedio de errores Cholesky

# Imprimir los resultados
print(f"Promedio total del error - LU: {lu_mean_total}")
print(f"Promedio total del error - PLU: {plu_mean_total}")
print(f"Promedio total del error - Cholesky: {cholesky_mean_total}")

# Comparar los promedios para ver cuál tiene mayor exactitud
if lu_mean_total < plu_mean_total and lu_mean_total < cholesky_mean_total:
    print("El algoritmo LU tiene la mayor exactitud (menor promedio de error).")
elif plu_mean_total < lu_mean_total and plu_mean_total < cholesky_mean_total:
    print("El algoritmo PLU tiene la mayor exactitud (menor promedio de error).")
else:
    print("El algoritmo Cholesky tiene la mayor exactitud (menor promedio de error).")


### Análisis de la Exactitud de los Algoritmos LU, PLU y Cholesky

La **exactitud** de un algoritmo se refiere a qué tan cercana es la solución obtenida a la solución teórica o exacta. Un **menor promedio de error** implica una mayor exactitud, ya que la diferencia con respecto a la solución teórica es mínima.

#### **Promedios de Error**:
- **LU**: 2.009312893552313e-09
- **PLU**: 4.639852032964125e-12
- **Cholesky**: 5.967195135740046e-09

#### **Análisis**:
- **PLU** es el algoritmo con la **mayor exactitud**, ya que tiene el menor promedio de error (4.639852032964125e-12). Esto indica que las soluciones que proporciona son las más cercanas a la solución teórica.
- **LU** tiene un promedio de error mayor que PLU, lo que significa que es **menos exacto**, ya que sus soluciones tienden a desviarse más de la solución teórica.
- **Cholesky** presenta el mayor promedio de error entre los tres algoritmos, lo que indica que es el **menos exacto**, con soluciones que se desvían significativamente de la solución teórica.

#### **Conclusión**:
- Basado en el promedio total del error, el algoritmo **PLU** es el más exacto, ya que proporciona las soluciones más cercanas a las soluciones teóricas. Los métodos **LU** y **Cholesky** son menos exactos, con Cholesky mostrando el mayor desvío respecto a la solución correcta.


## H)

In [235]:
# Calcular la desviación estándar total de los errores para cada algoritmo
lu_std_total = np.nanstd(lu_errors_matrix)  # Desviación estándar total para LU
plu_std_total = np.nanstd(plu_errors_matrix)  # Desviación estándar total para PLU
cholesky_std_total = np.nanstd(cholesky_errors_matrix)  # Desviación estándar total para Cholesky

# Imprimir los resultados
print(f"Desviación estándar total - LU: {lu_std_total}")
print(f"Desviación estándar total - PLU: {plu_std_total}")
print(f"Desviación estándar total - Cholesky: {cholesky_std_total}")

# Comparar los promedios para ver cuál tiene mayor precisión
if lu_std_total < plu_std_total and lu_std_total < cholesky_std_total:
    print("El algoritmo LU es el más preciso (menor desviación estándar promedio).")
elif plu_std_total < lu_std_total and plu_std_total < cholesky_std_total:
    print("El algoritmo PLU es el más preciso (menor desviación estándar promedio).")
else:
    print("El algoritmo Cholesky es el más preciso (menor desviación estándar promedio).")


### Análisis de la Precisión de los Algoritmos LU, PLU y Cholesky

La **precisión** de un algoritmo se refiere a qué tan repetitivas son las mediciones o soluciones obtenidas, independientemente de su exactitud.
 Una desviación estándar más pequeña indica una mayor precisión, ya que el error tiende a mantenerse constante entre diferentes ejecuciones.

#### **Promedios de Desviación Estándar**:
- **LU**: 5.384704256132522e-09
- **PLU**: 8.693358903865608e-14
- **Cholesky**: 4.512469881853291e-11

#### **Análisis**:
- **PLU** es el algoritmo con la **mayor precisión**, ya que tiene el promedio más bajo de desviación estándar. Esto implica que sus errores son muy consistentes y repetitivos en cada ejecución, lo que garantiza que el método es confiable en términos de producir resultados similares.
- **Cholesky** es el segundo más preciso, pero su desviación estándar promedio es significativamente mayor que PLU, lo que sugiere que hay más variabilidad en los errores.
- **LU** es el algoritmo **menos preciso** de los tres, con la desviación estándar promedio más alta. Esto significa que sus resultados son los menos consistentes y pueden variar considerablemente entre ejecuciones.

#### **Conclusión**:
- Basado en el promedio de la desviación estándar, el algoritmo **PLU** es el más preciso, ya que muestra la menor variabilidad en sus errores. Esto asegura que los resultados que genera son repetitivos y confiables.


# Estabilidad en LU

## A) y B)

In [226]:
# Definir las matrices de errores (25xlen(N)) preasignadas
lu_errors_matrix = np.zeros((25, len(N)))
plu_errors_matrix = np.zeros((25, len(N)))
# Definir el factor de escala para generar matrices aleatorias
scale = 25

# Iterar sobre cada dimensión n_i en N
for j, n in enumerate(N):
    for i in range(25):  # Generar 25 matrices aleatorias para cada dimensión n_i
        # Matriz aleatoria de dimensión n x n
        M = scale * np.random.rand(n, n)
        
        # Ajustar el segundo renglón de M para que sea igual al primero, excepto M[1,1]
        M[1, :] = M[0, :]  # Igualar el segundo renglón al primero
        
        # Escalar epsilon en función de M[0, 1]
        epsilon = 1e-15 * np.abs(M[0, 1])  
        M[1, 1] = M[0, 1] + epsilon  # Modificar la segunda entrada del segundo renglón
        
        # Verificar si la matriz es invertible antes de las factorizaciones
        if np.linalg.det(M) < 1e-10:  # Evitar matrices cercanas a ser singulares
            lu_errors_matrix[i, j] = np.nan
            plu_errors_matrix[i, j] = np.nan
            continue

        try:
            # Calcular factorizaciones y errores para LU
            L, U = lu_fact(M)
            lu_errors_matrix[i, j] = lu_error(M, L, U)
            
            # Calcular factorizaciones y errores para PLU
            P, L, U = plu_fact(M)
            plu_errors_matrix[i, j] = plu_error(M, P, L, U)
        
        except InvalidInputMatrix:
            # En caso de error, asignar NaN
            lu_errors_matrix[i, j] = np.nan
            plu_errors_matrix[i, j] = np.nan
# Calcular la media y desviación estándar para cada dimensión n_i
lu_mean = np.nanmean(lu_errors_matrix, axis=0)
lu_std = np.nanstd(lu_errors_matrix, axis=0)

plu_mean = np.nanmean(plu_errors_matrix, axis=0)
plu_std = np.nanstd(plu_errors_matrix, axis=0)


# Graficar la media de los errores con barras de error (desviación estándar)
plt.figure(figsize=(10, 6))

# Graficar las medias y desviaciones estándar de LU
plt.errorbar(N, lu_mean, yerr=lu_std, fmt='b--o', label='LU', markersize=6, linewidth=1.5)

# Graficar las medias y desviaciones estándar de PLU
plt.errorbar(N, plu_mean, yerr=plu_std, fmt='r--s', label='PLU', markersize=6, linewidth=1.5)

# Etiquetas de los ejes
plt.xlabel('$dim(M)=n_i$', fontsize=14)
plt.ylabel('Error Promedio (Norma de Frobenius)', fontsize=14)

# Título de la gráfica
plt.title('Media y Desviación Estándar de los Errores en Función de la Dimensión', fontsize=16)

# Mostrar la cuadrícula
plt.grid(alpha=0.6)

# Añadir la leyenda para identificar las curvas
plt.legend(fontsize=12)

# Mostrar la gráfica
plt.show()

# Imprimir los resultados para cada método
print("Media y desviación estándar de LU")
print(lu_mean)
print(lu_std)

print("\nMedia y desviación estándar de PLU")
print(plu_mean)
print(plu_std)



### Análisis Comparativo de la Media y Desviación Estándar entre LU y PLU

#### Gráfica de Media y Desviación Estándar de LU y PLU

A partir de los datos de **media de errores** y **desviación estándar**, podemos realizar un análisis de por qué el algoritmo **PLU** es más acertado que **LU**.

#### 1. **Exactitud (Media de errores)**:
- **PLU** presenta una media de errores consistentemente más baja que **LU**. Esto sugiere que las soluciones generadas por **PLU** son más cercanas a la solución teórica o exacta, es decir, **PLU es más exacto**.
- En comparación, la media de errores de **LU** es significativamente mayor, especialmente a medida que aumenta la dimensión, lo que indica que el algoritmo **LU tiende a producir soluciones menos precisas** conforme el problema se vuelve más grande y complejo.

#### 2. **Precisión (Desviación estándar)**:
- La **desviación estándar** en **PLU** es considerablemente más baja que en **LU**. Esto implica que **PLU es mucho más consistente** en los errores que produce, lo que significa que sus resultados son más predecibles y repetitivos.
- **LU**, por otro lado, muestra una desviación estándar mucho mayor, lo que indica una **alta variabilidad en los errores**. Esta falta de consistencia en las soluciones de LU sugiere que es menos confiable, especialmente en problemas de alta dimensión.

#### 3. **Comportamiento conforme aumenta la dimensión**:
- En ambas métricas (media y desviación estándar), **LU** presenta un crecimiento mucho más abrupto conforme aumenta la dimensión, lo que indica que su desempeño empeora de manera considerable a medida que el tamaño de la matriz aumenta.
- **PLU**, sin embargo, **mantiene tanto su media de errores como su desviación estándar bastante estables**, lo que refuerza su superioridad en términos de exactitud y precisión.

#### 4. **Conclusión**:
- **PLU es más acertado que LU** porque muestra un **menor promedio de errores** (mayor exactitud) y una **menor desviación estándar** (mayor precisión). Esto se debe a su capacidad para controlar mejor los errores numéricos mediante el uso del **pivoteo**, lo que hace que las soluciones obtenidas sean más cercanas a las correctas y más consistentes.
- **LU**, al no utilizar pivoteo, es más propenso a la amplificación de errores y genera soluciones menos predecibles y menos precisas, especialmente en problemas de mayor tamaño.
